In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

df = pd.read_parquet('./data')
print(f'total dataset: {len(df)} rows')

## filters
# df = df[df['stabilized_spread'] < 10]  # nem tul nagy spread
# df = df[df['stabilized_spread'] > 0.2]  # nem tul nagy spread
# df = df[abs(df['last_price_delta_since_stabilized']) > 0.1]
# df = df[abs(df['last_price_delta_since_stabilized']) < 8]

# df = df[df['stabilized_amount_mean'] < 0.4]  # nem tul nagy tradek
# df = df[df['stabilized_nr_trades'] < 300]  # nem tul sok trade

df.sort_index(axis=1, inplace=True)

y = df['last_price_delta_since_stabilized']
X = df.drop('last_price_delta_since_stabilized', axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=0)
X_train

total dataset: 70930 rows


,0_amount_mean,0_nr_trades,0_price_delta,0_spread,1_amount_mean,1_nr_trades,1_price_delta,1_spread,2_amount_mean,2_nr_trades,...,past_final_price_5,past_final_price_6,past_final_price_7,past_final_price_8,past_final_price_9,stabilized_amount_mean,stabilized_at_ms,stabilized_nr_trades,stabilized_spread,wave_direction
index,,,,,,,,,,,,,,,,,,,,,
0,0.0128,5.0,0.72,0.42,0.0082,8.0,0.72,0.71,0.0079,9.0,...,1.50,-5.79,-2.61,-0.05,-1.88,0.0065,361,13.0,1.43,max
0,0.0188,16.0,0.27,0.13,0.0183,17.0,0.27,0.13,0.0202,18.0,...,1.90,0.49,-3.93,-6.31,-7.08,0.0164,58,23.0,0.40,max
0,0.0115,4.0,0.00,0.94,0.0126,5.0,0.00,0.94,0.0109,6.0,...,9.68,13.54,10.97,8.40,13.60,0.0078,374,9.0,1.07,None
0,0.0053,3.0,0.93,0.13,0.0049,4.0,0.93,0.13,0.0055,5.0,...,-16.34,-14.38,-15.37,-14.97,-14.26,0.0109,100,12.0,1.08,min
0,0.0262,44.0,0.58,1.90,0.0257,45.0,0.58,1.90,0.0251,46.0,...,-15.79,-14.62,-13.03,-13.38,-17.86,0.0354,281,54.0,2.48,max
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.0346,72.0,0.31,3.25,0.0343,73.0,0.31,3.25,0.0338,74.0,...,9.32,11.37,8.83,7.65,10.74,0.0324,607,89.0,3.56,min
0,0.0009,6.0,0.42,1.09,0.0009,7.0,0.42,1.09,0.0010,8.0,...,4.27,3.27,4.57,4.18,6.34,0.0589,469,20.0,1.51,min
0,0.0054,4.0,0.00,0.84,0.0090,5.0,0.00,0.84,0.0078,6.0,...,2.98,5.15,6.35,7.26,7.49,0.0059,78,9.0,1.10,None


In [3]:
from spreadsurfer.price_engine import FeatureEngineer
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
# from sklearn.ensemle import RandomForestRegressor

model = CatBoostRegressor(learning_rate=0.015, depth=7, loss_function='MAPE', random_state=0, verbose=False, iterations=1500)
# model = CatBoostRegressor(learning_rate=0.015, depth=7, loss_function='RMSE', random_state=0, verbose=False, iterations=1500)
trained_model = model

pipeline = Pipeline(steps=[
    ('preprocessor', FeatureEngineer()),
    # ('scaler', StandardScaler()),
    # ('model', RandomForestRegressor(n_estimators=50, random_state=0))
    ('model', model)
])

pipeline.fit(X_train, y_train)
score = pipeline.score(X_valid, y_valid)
display(score)


2023-01-27 21:23:54.534 | INFO     | spreadsurfer.balance:<module>:6 - balance config: {'panic_below_total': 390, 'panic_countdown_from': 8}
2023-01-27 21:23:54.536 | INFO     | spreadsurfer.waves:<module>:14 - wave detection config: {'min_length': 6, 'investigate_length': 5, 'stabilized_threshold': 0.01, 'max_delta_ms_to_create_order': 500, 'collect_last_n_frames': 6, 'collect_last_n_wave_prices': 10}
2023-01-27 21:23:54.559 | INFO     | spreadsurfer.orders:<module>:17 - order config: {'orders_disabled': False, 'test_mode': True, 'max_nr_orders_limited': True, 'max_nr_orders_created': 20, 'base_amount': 0.0015, 'skip_order_on_spread_below': 0.2, 'skip_order_on_spread_above': 10, 'low_spread_limit': 0.5, 'recv_window': 100, 'cancel_far_order_after_ms': 400}


0.4441863436192153

In [517]:
import catboost
w = catboost.MetricVisualizer('./models', subdirs=True)
w.start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

boo


In [427]:
from sklearn.model_selection import GridSearchCV

do_grid_search = False

if do_grid_search:
    X = FeatureEngineer().transform(X)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

    parameters = {'depth': [7, 8],
                  'learning_rate': [0.015, 0.03, 0.04],
                  'iterations': [100, 1000, 1500]
                  }
    grid = GridSearchCV(estimator=model, param_grid=parameters, cv=2, n_jobs=-1)
    grid.fit(X_train, y_train)

    print(" Results from Grid Search ")
    print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
    print("\n The best score across ALL searched params:\n", grid.best_score_)
    print("\n The best parameters across ALL searched params:\n", grid.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:

 The best score across ALL searched params:
 0.5933380986132425

 The best parameters across ALL searched params:
 {'depth': 7, 'iterations': 1500, 'learning_rate': 0.015}


In [97]:
from spreadsurfer import now_isoformat

save = True
if save:
    model.save_model(f'./models/{now_isoformat()}-score-{1000 * round(score, 3)}.cat')


In [4]:
input_from_log = """

2023-01-27 18:42:22.014 | data     | spreadsurfer.datacollect:start:93 - wave 12629 collected: {'0_amount_mean': 0.0192, '0_nr_trades': 32.0, '0_price_delta': 0.23, '0_spread': 1.04, '1_amount_mean': 0.0187, '1_nr_trades': 33.0, '1_price_delta': 0.23, '1_spread': 1.04, '2_amount_mean': 0.0179, '2_nr_trades': 35.0, '2_price_delta': 0.23, '2_spread': 1.04, '3_amount_mean': 0.019, '3_nr_trades': 42.0, '3_price_delta': 0.22, '3_spread': 1.05, '4_amount_mean': 0.0188, '4_nr_trades': 43.0, '4_price_delta': 0.01, '4_spread': 1.26, 'last_price_delta_since_stabilized': -0.1, 'past_final_price_0': -5.869999999998981, 'past_final_price_1': -10.020000000000437, 'past_final_price_10': -9.2599999999984, 'past_final_price_11': -7.450000000000728, 'past_final_price_12': -9.700000000000728, 'past_final_price_13': -9.18999999999869, 'past_final_price_14': -6.3400000000001455, 'past_final_price_15': -8.479999999999563, 'past_final_price_16': -9.209999999999127, 'past_final_price_17': -6.5900000000001455, 'past_final_price_18': -6.170000000001892, 'past_final_price_19': -8.170000000001892, 'past_final_price_2': -8.790000000000873, 'past_final_price_20': -7.020000000000437, 'past_final_price_21': -6.170000000001892, 'past_final_price_22': -4.740000000001601, 'past_final_price_23': -0.2999999999992724, 'past_final_price_24': -2.4200000000018917, 'past_final_price_25': 3.400000000001455, 'past_final_price_26': -2.0, 'past_final_price_27': 0.5600000000013097, 'past_final_price_28': -0.27999999999883585, 'past_final_price_29': 0.0, 'past_final_price_3': -9.389999999999418, 'past_final_price_4': -11.220000000001164, 'past_final_price_5': -7.619999999998981, 'past_final_price_6': -6.130000000001019, 'past_final_price_7': -7.380000000001019, 'past_final_price_8': -8.049999999999272, 'past_final_price_9': -11.080000000001746, 'stabilized_amount_mean': 0.0186, 'stabilized_at_ms': 622, 'stabilized_nr_trades': 44.0, 'stabilized_spread': 1.27, 'wave_direction': 'max'}
2023-01-27 18:42:22.628 | data     | spreadsurfer.datacollect:start:93 - wave 12630 collected: {'0_amount_mean': 0.0015, '0_nr_trades': 3.0, '0_price_delta': 0.36, '0_spread': 0.02, '1_amount_mean': 0.0016, '1_nr_trades': 4.0, '1_price_delta': 0.0, '1_spread': 0.38, '2_amount_mean': 0.0015, '2_nr_trades': 5.0, '2_price_delta': 0.0, '2_spread': 0.41, '3_amount_mean': 0.0021, '3_nr_trades': 6.0, '3_price_delta': 0.0, '3_spread': 0.41, '4_amount_mean': 0.0019, '4_nr_trades': 7.0, '4_price_delta': 0.0, '4_spread': 0.41, 'last_price_delta_since_stabilized': -0.51, 'past_final_price_0': -5.869999999998981, 'past_final_price_1': -10.020000000000437, 'past_final_price_10': -9.2599999999984, 'past_final_price_11': -7.450000000000728, 'past_final_price_12': -9.700000000000728, 'past_final_price_13': -9.18999999999869, 'past_final_price_14': -6.3400000000001455, 'past_final_price_15': -8.479999999999563, 'past_final_price_16': -9.209999999999127, 'past_final_price_17': -6.5900000000001455, 'past_final_price_18': -6.170000000001892, 'past_final_price_19': -8.170000000001892, 'past_final_price_2': -8.790000000000873, 'past_final_price_20': -7.020000000000437, 'past_final_price_21': -6.170000000001892, 'past_final_price_22': -4.740000000001601, 'past_final_price_23': -0.2999999999992724, 'past_final_price_24': -2.4200000000018917, 'past_final_price_25': 3.400000000001455, 'past_final_price_26': -2.0, 'past_final_price_27': 0.5600000000013097, 'past_final_price_28': -0.27999999999883585, 'past_final_price_29': 0.0, 'past_final_price_3': -9.389999999999418, 'past_final_price_4': -11.220000000001164, 'past_final_price_5': -7.619999999998981, 'past_final_price_6': -6.130000000001019, 'past_final_price_7': -7.380000000001019, 'past_final_price_8': -8.049999999999272, 'past_final_price_9': -11.080000000001746, 'stabilized_amount_mean': 0.0017, 'stabilized_at_ms': 255, 'stabilized_nr_trades': 8.0, 'stabilized_spread': 0.42, 'wave_direction': None}
2023-01-27 18:42:23.017 | data     | spreadsurfer.datacollect:start:93 - wave 12631 collected: {'0_amount_mean': 0.0019, '0_nr_trades': 7.0, '0_price_delta': 0.57, '0_spread': 0.41, '1_amount_mean': 0.0017, '1_nr_trades': 8.0, '1_price_delta': 0.57, '1_spread': 0.42, '2_amount_mean': 0.0016, '2_nr_trades': 9.0, '2_price_delta': 0.57, '2_spread': 0.42, '3_amount_mean': 0.0018, '3_nr_trades': 10.0, '3_price_delta': 0.57, '3_spread': 0.42, '4_amount_mean': 0.0019, '4_nr_trades': 11.0, '4_price_delta': 0.06, '4_spread': 0.93, 'last_price_delta_since_stabilized': -2.24, 'past_final_price_0': -11.880000000001019, 'past_final_price_1': -10.650000000001455, 'past_final_price_10': -9.31000000000131, 'past_final_price_11': -11.56000000000131, 'past_final_price_12': -11.049999999999272, 'past_final_price_13': -8.200000000000728, 'past_final_price_14': -10.340000000000146, 'past_final_price_15': -11.069999999999709, 'past_final_price_16': -8.450000000000728, 'past_final_price_17': -8.030000000002474, 'past_final_price_18': -10.030000000002474, 'past_final_price_19': -8.880000000001019, 'past_final_price_2': -11.25, 'past_final_price_20': -8.030000000002474, 'past_final_price_21': -6.600000000002183, 'past_final_price_22': -2.1599999999998545, 'past_final_price_23': -4.280000000002474, 'past_final_price_24': 1.5400000000008731, 'past_final_price_25': -3.860000000000582, 'past_final_price_26': -1.2999999999992724, 'past_final_price_27': -2.139999999999418, 'past_final_price_28': -1.860000000000582, 'past_final_price_29': 0.0, 'past_final_price_3': -13.080000000001746, 'past_final_price_4': -9.479999999999563, 'past_final_price_5': -7.990000000001601, 'past_final_price_6': -9.2400000000016, 'past_final_price_7': -9.909999999999854, 'past_final_price_8': -12.940000000002328, 'past_final_price_9': -11.119999999998981, 'stabilized_amount_mean': 0.0018, 'stabilized_at_ms': 458, 'stabilized_nr_trades': 12.0, 'stabilized_spread': 0.99, 'wave_direction': 'max'}
2023-01-27 18:42:24.090 | data     | spreadsurfer.datacollect:start:93 - wave 12632 collected: {'0_amount_mean': 0.0184, '0_nr_trades': 8.0, '0_price_delta': 0.04, '0_spread': 0.4, '1_amount_mean': 0.0165, '1_nr_trades': 9.0, '1_price_delta': 0.03, '1_spread': 0.41, '2_amount_mean': 0.0224, '2_nr_trades': 10.0, '2_price_delta': 0.03, '2_spread': 0.41, '3_amount_mean': 0.0204, '3_nr_trades': 11.0, '3_price_delta': 0.03, '3_spread': 0.41, '4_amount_mean': 0.0188, '4_nr_trades': 12.0, '4_price_delta': 0.03, '4_spread': 0.41, 'last_price_delta_since_stabilized': 0.84, 'past_final_price_0': -8.900000000001455, 'past_final_price_1': -9.5, 'past_final_price_10': -9.81000000000131, 'past_final_price_11': -9.299999999999272, 'past_final_price_12': -6.450000000000728, 'past_final_price_13': -8.590000000000146, 'past_final_price_14': -9.319999999999709, 'past_final_price_15': -6.700000000000728, 'past_final_price_16': -6.280000000002474, 'past_final_price_17': -8.280000000002474, 'past_final_price_18': -7.130000000001019, 'past_final_price_19': -6.280000000002474, 'past_final_price_2': -11.330000000001746, 'past_final_price_20': -4.850000000002183, 'past_final_price_21': -0.4099999999998545, 'past_final_price_22': -2.530000000002474, 'past_final_price_23': 3.290000000000873, 'past_final_price_24': -2.110000000000582, 'past_final_price_25': 0.4500000000007276, 'past_final_price_26': -0.3899999999994179, 'past_final_price_27': -0.11000000000058208, 'past_final_price_28': 1.75, 'past_final_price_29': 0.0, 'past_final_price_3': -7.729999999999563, 'past_final_price_4': -6.240000000001601, 'past_final_price_5': -7.490000000001601, 'past_final_price_6': -8.159999999999854, 'past_final_price_7': -11.190000000002328, 'past_final_price_8': -9.369999999998981, 'past_final_price_9': -7.56000000000131, 'stabilized_amount_mean': 0.0174, 'stabilized_at_ms': 683, 'stabilized_nr_trades': 13.0, 'stabilized_spread': 0.44, 'wave_direction': 'min'}
2023-01-27 18:42:24.555 | data     | spreadsurfer.datacollect:start:93 - wave 12633 collected: {'0_amount_mean': 0.0011, '0_nr_trades': 3.0, '0_price_delta': 0.91, '0_spread': 0.27, '1_amount_mean': 0.002, '1_nr_trades': 4.0, '1_price_delta': 0.79, '1_spread': 0.39, '2_amount_mean': 0.0116, '2_nr_trades': 5.0, '2_price_delta': 0.79, '2_spread': 0.39, '3_amount_mean': 0.0099, '3_nr_trades': 6.0, '3_price_delta': 0.79, '3_spread': 0.39, '4_amount_mean': 0.0094, '4_nr_trades': 13.0, '4_price_delta': 0.0, '4_spread': 1.18, 'last_price_delta_since_stabilized': -0.11, 'past_final_price_0': -8.900000000001455, 'past_final_price_1': -9.5, 'past_final_price_10': -9.81000000000131, 'past_final_price_11': -9.299999999999272, 'past_final_price_12': -6.450000000000728, 'past_final_price_13': -8.590000000000146, 'past_final_price_14': -9.319999999999709, 'past_final_price_15': -6.700000000000728, 'past_final_price_16': -6.280000000002474, 'past_final_price_17': -8.280000000002474, 'past_final_price_18': -7.130000000001019, 'past_final_price_19': -6.280000000002474, 'past_final_price_2': -11.330000000001746, 'past_final_price_20': -4.850000000002183, 'past_final_price_21': -0.4099999999998545, 'past_final_price_22': -2.530000000002474, 'past_final_price_23': 3.290000000000873, 'past_final_price_24': -2.110000000000582, 'past_final_price_25': 0.4500000000007276, 'past_final_price_26': -0.3899999999994179, 'past_final_price_27': -0.11000000000058208, 'past_final_price_28': 1.75, 'past_final_price_29': 0.0, 'past_final_price_3': -7.729999999999563, 'past_final_price_4': -6.240000000001601, 'past_final_price_5': -7.490000000001601, 'past_final_price_6': -8.159999999999854, 'past_final_price_7': -11.190000000002328, 'past_final_price_8': -9.369999999998981, 'past_final_price_9': -7.56000000000131, 'stabilized_amount_mean': 0.0095, 'stabilized_at_ms': 408, 'stabilized_nr_trades': 14.0, 'stabilized_spread': 1.18, 'wave_direction': None}
2023-01-27 18:42:25.024 | data     | spreadsurfer.datacollect:start:93 - wave 12634 collected: {'0_amount_mean': 0.0089, '0_nr_trades': 15.0, '0_price_delta': 0.68, '0_spread': 1.19, '1_amount_mean': 0.0085, '1_nr_trades': 16.0, '1_price_delta': 0.68, '1_spread': 1.26, '2_amount_mean': 0.0127, '2_nr_trades': 20.0, '2_price_delta': 0.0, '2_spread': 1.97, '3_amount_mean': 0.0122, '3_nr_trades': 21.0, '3_price_delta': 0.0, '3_spread': 1.97, '4_amount_mean': 0.0118, '4_nr_trades': 22.0, '4_price_delta': 0.0, '4_spread': 1.97, 'last_price_delta_since_stabilized': 0.0, 'past_final_price_0': -10.279999999998836, 'past_final_price_1': -12.110000000000582, 'past_final_price_10': -10.079999999998108, 'past_final_price_11': -7.229999999999563, 'past_final_price_12': -9.369999999998981, 'past_final_price_13': -10.099999999998545, 'past_final_price_14': -7.479999999999563, 'past_final_price_15': -7.06000000000131, 'past_final_price_16': -9.06000000000131, 'past_final_price_17': -7.9099999999998545, 'past_final_price_18': -7.06000000000131, 'past_final_price_19': -5.630000000001019, 'past_final_price_2': -8.5099999999984, 'past_final_price_20': -1.1899999999986903, 'past_final_price_21': -3.3100000000013097, 'past_final_price_22': 2.5100000000020373, 'past_final_price_23': -2.889999999999418, 'past_final_price_24': -0.32999999999810825, 'past_final_price_25': -1.1699999999982538, 'past_final_price_26': -0.8899999999994179, 'past_final_price_27': 0.9700000000011642, 'past_final_price_28': -0.7799999999988358, 'past_final_price_29': 0.0, 'past_final_price_3': -7.020000000000437, 'past_final_price_4': -8.270000000000437, 'past_final_price_5': -8.93999999999869, 'past_final_price_6': -11.970000000001164, 'past_final_price_7': -10.149999999997817, 'past_final_price_8': -8.340000000000146, 'past_final_price_9': -10.590000000000146, 'stabilized_amount_mean': 0.013, 'stabilized_at_ms': 808, 'stabilized_nr_trades': 24.0, 'stabilized_spread': 1.97, 'wave_direction': 'min'}
2023-01-27 18:42:26.085 | data     | spreadsurfer.datacollect:start:93 - wave 12635 collected: {'0_amount_mean': 0.0036, '0_nr_trades': 3.0, '0_price_delta': 1.13, '0_spread': 0.44, '1_amount_mean': 0.0032, '1_nr_trades': 4.0, '1_price_delta': 0.38, '1_spread': 1.19, '2_amount_mean': 0.003, '2_nr_trades': 5.0, '2_price_delta': 0.38, '2_spread': 1.19, '3_amount_mean': 0.0031, '3_nr_trades': 6.0, '3_price_delta': 0.38, '3_spread': 1.19, '4_amount_mean': 0.0084, '4_nr_trades': 7.0, '4_price_delta': 0.38, '4_spread': 1.19, 'last_price_delta_since_stabilized': 1.14, 'past_final_price_0': -11.540000000000873, 'past_final_price_1': -7.93999999999869, 'past_final_price_10': -6.6599999999998545, 'past_final_price_11': -8.799999999999272, 'past_final_price_12': -9.529999999998836, 'past_final_price_13': -6.9099999999998545, 'past_final_price_14': -6.490000000001601, 'past_final_price_15': -8.4900000000016, 'past_final_price_16': -7.3400000000001455, 'past_final_price_17': -6.490000000001601, 'past_final_price_18': -5.06000000000131, 'past_final_price_19': -0.6199999999989814, 'past_final_price_2': -6.450000000000728, 'past_final_price_20': -2.7400000000016007, 'past_final_price_21': 3.0800000000017462, 'past_final_price_22': -2.319999999999709, 'past_final_price_23': 0.2400000000016007, 'past_final_price_24': -0.5999999999985448, 'past_final_price_25': -0.31999999999970896, 'past_final_price_26': 1.5400000000008731, 'past_final_price_27': -0.20999999999912689, 'past_final_price_28': 0.569999999999709, 'past_final_price_29': 0.0, 'past_final_price_3': -7.700000000000728, 'past_final_price_4': -8.369999999998981, 'past_final_price_5': -11.400000000001455, 'past_final_price_6': -9.579999999998108, 'past_final_price_7': -7.770000000000437, 'past_final_price_8': -10.020000000000437, 'past_final_price_9': -9.5099999999984, 'stabilized_amount_mean': 0.0079, 'stabilized_at_ms': 243, 'stabilized_nr_trades': 8.0, 'stabilized_spread': 1.57, 'wave_direction': 'min'}
2023-01-27 18:42:27.278 | data     | spreadsurfer.datacollect:start:93 - wave 12636 collected: {'0_amount_mean': 0.0708, '0_nr_trades': 4.0, '0_price_delta': 1.15, '0_spread': 0.26, '1_amount_mean': 0.0569, '1_nr_trades': 5.0, '1_price_delta': 1.15, '1_spread': 0.75, '2_amount_mean': 0.0475, '2_nr_trades': 6.0, '2_price_delta': 1.15, '2_spread': 0.77, '3_amount_mean': 0.0358, '3_nr_trades': 8.0, '3_price_delta': 0.0, '3_spread': 1.92, '4_amount_mean': 0.0406, '4_nr_trades': 10.0, '4_price_delta': 0.0, '4_spread': 1.92, 'last_price_delta_since_stabilized': 0.08, 'past_final_price_0': -7.959999999999127, 'past_final_price_1': -6.470000000001164, 'past_final_price_10': -8.819999999999709, 'past_final_price_11': -9.549999999999272, 'past_final_price_12': -6.930000000000291, 'past_final_price_13': -6.510000000002037, 'past_final_price_14': -8.510000000002037, 'past_final_price_15': -7.360000000000582, 'past_final_price_16': -6.510000000002037, 'past_final_price_17': -5.080000000001746, 'past_final_price_18': -0.6399999999994179, 'past_final_price_19': -2.7600000000020373, 'past_final_price_2': -7.720000000001164, 'past_final_price_20': 3.0600000000013097, 'past_final_price_21': -2.3400000000001455, 'past_final_price_22': 0.22000000000116415, 'past_final_price_23': -0.6199999999989814, 'past_final_price_24': -0.3400000000001455, 'past_final_price_25': 1.5200000000004366, 'past_final_price_26': -0.22999999999956344, 'past_final_price_27': 0.5499999999992724, 'past_final_price_28': -0.020000000000436557, 'past_final_price_29': 0.0, 'past_final_price_3': -8.389999999999418, 'past_final_price_4': -11.420000000001892, 'past_final_price_5': -9.599999999998545, 'past_final_price_6': -7.790000000000873, 'past_final_price_7': -10.040000000000873, 'past_final_price_8': -9.529999999998836, 'past_final_price_9': -6.680000000000291, 'stabilized_amount_mean': 0.0412, 'stabilized_at_ms': 120, 'stabilized_nr_trades': 11.0, 'stabilized_spread': 1.92, 'wave_direction': 'min'}
2023-01-27 18:42:30.015 | data     | spreadsurfer.datacollect:start:93 - wave 12637 collected: {'0_amount_mean': 0.0273, '0_nr_trades': 57.0, '0_price_delta': 0.25, '0_spread': 1.95, '1_amount_mean': 0.0394, '1_nr_trades': 58.0, '1_price_delta': 0.25, '1_spread': 1.95, '2_amount_mean': 0.0388, '2_nr_trades': 59.0, '2_price_delta': 0.25, '2_spread': 1.95, '3_amount_mean': 0.0381, '3_nr_trades': 60.0, '3_price_delta': 0.25, '3_spread': 1.95, '4_amount_mean': 0.0369, '4_nr_trades': 62.0, '4_price_delta': 0.25, '4_spread': 1.95, 'last_price_delta_since_stabilized': -0.43, 'past_final_price_0': -11.290000000000873, 'past_final_price_1': -14.320000000003347, 'past_final_price_10': -9.410000000003492, 'past_final_price_11': -11.410000000003492, 'past_final_price_12': -10.260000000002037, 'past_final_price_13': -9.410000000003492, 'past_final_price_14': -7.980000000003201, 'past_final_price_15': -3.540000000000873, 'past_final_price_16': -5.6600000000034925, 'past_final_price_17': 0.15999999999985448, 'past_final_price_18': -5.240000000001601, 'past_final_price_19': -2.680000000000291, 'past_final_price_2': -12.5, 'past_final_price_20': -3.5200000000004366, 'past_final_price_21': -3.2400000000016007, 'past_final_price_22': -1.3800000000010186, 'past_final_price_23': -3.1300000000010186, 'past_final_price_24': -2.350000000002183, 'past_final_price_25': -2.9200000000018917, 'past_final_price_26': -2.900000000001455, 'past_final_price_27': -1.640000000003056, 'past_final_price_28': -2.6300000000010186, 'past_final_price_29': 0.0, 'past_final_price_3': -10.690000000002328, 'past_final_price_4': -12.940000000002328, 'past_final_price_5': -12.430000000000291, 'past_final_price_6': -9.580000000001746, 'past_final_price_7': -11.720000000001164, 'past_final_price_8': -12.450000000000728, 'past_final_price_9': -9.830000000001746, 'stabilized_amount_mean': 0.0333, 'stabilized_at_ms': 347, 'stabilized_nr_trades': 70.0, 'stabilized_spread': 2.2, 'wave_direction': 'max'}


"""
from timeit import timeit

def predict_samples():
    mae = 0
    count = 0
    for line in input_from_log.split('\n'):
        if not line: continue
        count += 1
        json = '{' + line.split('{')[1]
        sample = pd.DataFrame([eval(json)])

        real = sample.last_price_delta_since_stabilized[0]
        print('expected: ', real)
        sample.drop('last_price_delta_since_stabilized', axis=1, inplace=True)

        timeit(lambda: pipeline.predict(sample), number=1) * 1000
        guess = pipeline.predict(sample)[0]
        print('predict: ', guess)
        mae += abs(real - guess)
        print()
    display(f'MAE: {mae / count}')
predict_samples()

expected:  -0.1
predict:  -0.3199425776286879

expected:  -0.51
predict:  -0.20278890487302645

expected:  -2.24
predict:  -0.3199425776286879

expected:  0.84
predict:  0.4185761323424823

expected:  -0.11
predict:  -0.02184898017046554

expected:  0.0
predict:  -0.01874480109696403

expected:  1.14
predict:  0.15890465347508517

expected:  0.08
predict:  0.11944377829551847

expected:  -0.43
predict:  -0.3199425776286879



'MAE: 0.45623637010030393'

In [100]:
sample = pd.DataFrame([
    {'0_amount_mean': 0.007, '0_nr_trades': 12.0, '0_price_delta': 0.91, '0_spread': 1.15, '1_amount_mean': 0.007, '1_nr_trades': 13.0, '1_price_delta': 0.91, '1_spread': 1.52, '2_amount_mean': 0.006, '2_nr_trades': 14.0,
     '2_price_delta': 0.91, '2_spread': 1.52, '3_amount_mean': 0.006, '3_nr_trades': 16.0, '3_price_delta': 0.69, '3_spread': 1.74, '4_amount_mean': 0.005, '4_nr_trades': 17.0, '4_price_delta': 0.69, '4_spread': 1.74,
     'last_price_delta_since_stabilized': -0.87, 'past_final_price_0': -0.11999999999898137, 'past_final_price_1': 0.25, 'past_final_price_2': -0.020000000000436557, 'past_final_price_3': -1.7400000000016007,
     'past_final_price_4': 0.06999999999970896, 'past_final_price_5': -2.7299999999995634, 'past_final_price_6': -3.9099999999998545, 'past_final_price_7': 3.360000000000582, 'past_final_price_8': -0.06000000000130967,
     'past_final_price_9': 0.0, 'stabilized_amount_mean': 0.007, 'stabilized_at_ms': 369, 'stabilized_nr_trades': 18.0, 'stabilized_spread': 2.43, 'wave_direction': 'max'}
])

print('expected: ', sample.last_price_delta_since_stabilized.real())
sample.drop('last_price_delta_since_stabilized', axis=1, inplace=True)

from timeit import timeit

timeit(lambda: pipeline.predict(sample), number=1) * 1000
print('predict: ', pipeline.predict(sample)[0])


AttributeError: 'Series' object has no attribute 'real'

In [20]:
from glob import glob

cat_filename = glob('*.cat')[0]
cat_filename

model = CatBoostRegressor()
model.load_model(fname=cat_filename)
print(f'catboost model loaded from {cat_filename}')


pipeline = Pipeline(steps=[
    ('preprocessor', FeatureEngineer()),
    ('model', model)
])

predict_samples()


catboost model loaded from 2023-01-24T07:03:25-score-530.0.cat
expected:  -1.03
predict:  -1.0136278914883838

expected:  0.0
predict:  4.358048706401518

expected:  1.45
predict:  5.4119201464101145

expected:  0.0
predict:  7.42561778068513

expected:  -0.1
predict:  7.237852844753644

expected:  -0.11
predict:  6.7176724009602875

expected:  0.29
predict:  6.259447424454364

expected:  -0.02
predict:  -1.0136278914883838

expected:  -0.27
predict:  2.198350519988562

expected:  0.0
predict:  -1.0136278914883838



'MAE: 4.037253771514201'